![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Strategy 1: Buy and hold

Buy and hold generates profit from the risk premia of market risk exposition.

A buy and hold strategy of 1 instrument is the most simple strategy. It requires several considerations that will be touched on in this notebook

# Data aquisition

In [41]:
from datetime import datetime
from QuantConnect import Resolution
from QuantConnect.Securities import Futures
from helper import calculate_return, calculate_percentage_return, calculate_transaction_costs, expiration_dates, calculate_and_merge_returns, summarize_statistics, calculate_contract_risk
import numpy as np
from scipy.stats import skew

In [42]:
# instruments = {
#     'ES': {
#         'multiplier': 50,
#         'tick_value': 0.25,
#         'minimum_fluctuation': 12.50,
#         'spread': 0.25,
#         'commission': 2.50,
#         'expiration_dates': [3, 6, 9, 12]
#     },
#     'MES': {
#         'multiplier': 5,
#         'tick_value': 0.25,
#         'minimum_fluctuation': 1.25,
#         'spread': 0.25,
#         'commission': 2.50,
#         'expiration_dates': [3, 6, 9, 12]
#     },
#     'ZN': {  # 10-Year Treasury
#         'multiplier': 1000,
#         'tick_value': 0.015625,
#         'minimum_fluctuation': 15.625,
#         'spread': 0.015625,
#         'commission': 2.50,
#         'expiration_dates': [6, 9, 12]  # Placeholder
#     },
#     'ZF': {  # 5-Year Treasury
#         'multiplier': 1000,
#         'tick_value': 0.0078125,
#         'minimum_fluctuation': 7.8125,
#         'spread': 0.0078125,
#         'commission': 2.50,
#         'expiration_dates': [6, 9, 12]  # Placeholder
#     },
#     'ZB': {  # 30-Year Treasury
#         'multiplier': 1000,
#         'tick_value': 0.015625,
#         'minimum_fluctuation': 15.625,
#         'spread': 0.015625,
#         'commission': 2.50,
#         'expiration_dates': [6, 9, 12]  # Placeholder
#     },
#     'ZW': {  # Wheat
#         'multiplier': 50,
#         'tick_value': 0.0025,
#         'minimum_fluctuation': 12.50,
#         'spread': 0.0025,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'CL': {  # Crude Oil
#         'multiplier': 1000,
#         'tick_value': 0.01,
#         'minimum_fluctuation': 10.00,
#         'spread': 0.01,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'GC': {  # Gold
#         'multiplier': 100,
#         'tick_value': 0.1,
#         'minimum_fluctuation': 10.00,
#         'spread': 0.1,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'HE': {  # Lean Hog
#         'multiplier': 400,
#         'tick_value': 0.00025,
#         'minimum_fluctuation': 10.00,
#         'spread': 0.00025,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'KC': {  # Coffee
#         'multiplier': 375,
#         'tick_value': 0.0005,
#         'minimum_fluctuation': 18.75,
#         'spread': 0.0005,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#         'SB': {  # Sugar
#         'multiplier': 1120,
#         'tick_value': 0.0001,
#         'minimum_fluctuation': 11.20,
#         'spread': 0.0001,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'GF': {  # Feeder Cattle
#         'multiplier': 500,
#         'tick_value': 0.00025,
#         'minimum_fluctuation': 12.50,
#         'spread': 0.00025,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'ZS': {  # Soy Bean
#         'multiplier': 50,
#         'tick_value': 0.0025,
#         'minimum_fluctuation': 12.50,
#         'spread': 0.0025,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'ZC': {  # Corn
#         'multiplier': 50,
#         'tick_value': 0.0025,
#         'minimum_fluctuation': 12.50,
#         'spread': 0.0025,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'NG': {  # Natural Gas
#         'multiplier': 10000,
#         'tick_value': 0.001,
#         'minimum_fluctuation': 10.00,
#         'spread': 0.001,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'SI': {  # Silver
#         'multiplier': 5000,
#         'tick_value': 0.005,
#         'minimum_fluctuation': 25.00,
#         'spread': 0.005,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'HG': {  # Copper
#         'multiplier': 25000,
#         'tick_value': 0.0005,
#         'minimum_fluctuation': 12.50,
#         'spread': 0.0005,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'NQ': {  # Nasdaq
#         'multiplier': 20,
#         'tick_value': 0.25,
#         'minimum_fluctuation': 5.00,
#         'spread': 0.25,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     },
#     'CT': {  # Cotton
#         'multiplier': 500,
#         'tick_value': 0.0001,
#         'minimum_fluctuation': 5.00,
#         'spread': 0.0001,
#         'commission': 2.50,
#         'expiration_dates': []  # Placeholder
#     }
# }



In [43]:
# Define Instruments Dictionary
instruments = {
    'ES': {
        'multiplier': 50,
        'tick_value': 0.25,
        'minimum_fluctuation': 12.50,
        'spread': 0.25,  # Typical spread
        'commission': 2.50,  # Typical commission per contract
        'expiration_dates': expiration_dates['ES'],
        'subscription_type': 'Future',
        'subscription_details': Futures.Indices.SP500EMini,
        'resolution': Resolution.Daily
    },
    'MES': {
        'multiplier': 5,
        'tick_value': 0.25,
        'minimum_fluctuation': 1.25,
        'spread': 0.25,  # Typical spread
        'commission': 2.50,  # Typical commission per contract
        'expiration_dates': expiration_dates['MES'],
        'subscription_type': 'Future',
        'subscription_details': Futures.Indices.MicroSP500EMini,
        'resolution': Resolution.Daily
    },
    'USDMXN': {
        'subscription_type': 'Forex',
        'subscription_details': 'USDMXN',
        'resolution': Resolution.Daily
    }
}



In [44]:
from QuantConnect.Data.UniverseSelection import *
from datetime import datetime

# Initialize QuantBook
qb = QuantBook()

# Add Subscriptions for Futures and Forex
subscriptions = {}
for symbol, details in instruments.items():
    if details['subscription_type'] == 'Future':
        future = qb.AddFuture(details['subscription_details'], details['resolution'])
        subscriptions[symbol] = future.Symbol
    elif details['subscription_type'] == 'Forex':
        forex = qb.AddForex(details['subscription_details'], details['resolution'])
        subscriptions[symbol] = forex.Symbol

# Set Historical Data Range
start_date = datetime(2007, 1, 1)
end_date = datetime.now()
trading_days_per_year = 256
capital = 100000
target_risk = .20

# Request Historical Data
timeseries = {}
for symbol, sub_symbol in subscriptions.items():
    history = qb.History([sub_symbol], start_date, end_date, instruments[symbol]['resolution'])
    if instruments[symbol]['subscription_type'] == 'Future':
        history.reset_index(level=0, drop=True, inplace=True)
    timeseries[symbol] = history[['close']]

del details
del end_date
del forex
del future
del history
del start_date
del sub_symbol
del symbol


## Print the timeseries

In [45]:
#Print symbols and first 5 entires of historical_data
for symbol, data in timeseries.items():
    print(symbol)
    print(data.head())

ES
                                  close
symbol time                            
ES 1S1 2007-01-03 19:00:00  1380.201123
       2007-01-04 19:00:00  1383.106300
       2007-01-05 19:00:00  1371.727690
       2007-01-08 19:00:00  1378.022240
       2007-01-09 19:00:00  1376.327554
MES
                                   close
symbol  time                            
MES 1S1 2019-05-06 20:00:00  3044.041911
        2019-05-07 20:00:00  3024.166723
        2019-05-08 20:00:00  3017.367317
        2019-05-09 20:00:00  3004.291536
        2019-05-10 20:00:00  3020.243989
USDMXN
                                 close
symbol    time                        
USDMXN 8G 2007-01-01 19:00:00  10.8070
          2007-01-02 19:00:00  10.7710
          2007-01-03 19:00:00  10.8338
          2007-01-04 19:00:00  10.8930
          2007-01-05 19:00:00  10.9428


## Individual dataframe creation

In [29]:

for symbol, data in timeseries.items():
    globals()[symbol] = pd.DataFrame(data)


## Summary statistics
We can begin working with the aquired data to calculate some useful statistics, such as returns, standard deviations, skew and fat tail ratios. Use the helper function to process these statistics.

### Returns Time Series

In [46]:
# Added to helper.py, delete next time

def calculate_and_merge_returns(symbol, price_series, instruments):
    """
    Inputs 
    symbol: str - a symbol to calculate returns for
    price_series: pd.Series - a price series to calculate returns from
    instruments: dict - a dictionary of instruments with metadata

    Returns
    merged_df: pd.DataFrame - a DataFrame with close, return, and percentage return columns
    """
    base_returns = calculate_return(symbol, price_series, instruments)
    percentage_returns = calculate_percentage_return(symbol, price_series, instruments, num_contracts=1)
    
    # Merge the DataFrames based on the index
    merged_df = base_returns.join(percentage_returns.rename('percentage_return'))
    # Include the original close price series
    merged_df['close'] = price_series
    
    return merged_df


In [47]:

# Computes returns in price points, notional value, and percentage
for symbol, price_df in timeseries.items():
    price_series = price_df['close']
    merged_df = calculate_and_merge_returns(symbol, price_series, instruments)
    timeseries[symbol] = merged_df.loc[:, ['close', 'price_return', 'currency_return', 'percentage_return']]

del price_df, price_series, merged_df


### Summary Stats for each symbol

In [48]:
summary = summarize_statistics(timeseries, trading_days_per_year)
print("Summary Statistics for Each Symbol:")

for symbol, stats in summary.items():
    print(f"{symbol}:")
    print(summary[symbol])

    


Summary Statistics for Each Symbol:
ES:
{'mean_return': 0.038056750980877456, 'std_dev': 1.2504315625779854, 'annualized_mean': 9.742528251104629, 'annualized_std_dev': 20.006905001247766, 'daily_sharpe_ratio': 0.030434893135947997, 'annualized_sharpe_ratio': 0.48695829017516795, 'monthly_skew': -14.351501476816775, 'lower_fat_tail_ratio': 2.5680685342096266, 'higher_fat_tail_ratio': 1.7483440973176516}
MES:
{'mean_return': 0.051487616191232134, 'std_dev': 1.3017453243703707, 'annualized_mean': 13.180829744955426, 'annualized_std_dev': 20.82792518992593, 'daily_sharpe_ratio': 0.039552756769943236, 'annualized_sharpe_ratio': 0.6328441083190918, 'monthly_skew': -7.616867247540757, 'lower_fat_tail_ratio': 2.125288636742835, 'higher_fat_tail_ratio': 1.4225590023635961}
USDMXN:
{'mean_return': 0.010734745383597002, 'std_dev': 0.7347118272826658, 'annualized_mean': 2.7480948182008325, 'annualized_std_dev': 11.755389236522653, 'daily_sharpe_ratio': 0.014610824251052952, 'annualized_sharpe_rat

## Backtest buy and hold 1 contract. Return is a dataframe or time series.

In [49]:

# Definir la función de simulación
def simulate_buy_and_hold(history_df, symbol, fx_rates_df, num_contracts=1):
    params = instruments[symbol]
    initial_price = history_df['close'].iloc[0]
    final_price = history_df['close'].iloc[-1]

    total_return_in_usd = calculate_return_in_usd(symbol, initial_price, final_price, instruments)
    total_return_in_mxn = calculate_return_in_mxn(total_return_in_usd, fx_rates_df)

    # Calcular costos de transacción
    transaction_costs = calculate_transaction_costs(params, history_df, start_date, end_date)

    # Ajustar retornos por costos de transacción
    net_return_in_usd = total_return_in_usd - transaction_costs
    net_return_in_mxn = calculate_return_in_mxn(net_return_in_usd, fx_rates_df)

    # Multiplicar los retornos por el número de contratos
    net_return_in_usd *= num_contracts
    net_return_in_mxn *= num_contracts

    return {
        "return_in_points": (final_price - initial_price) * num_contracts,
        "return_in_usd": net_return_in_usd,
        "return_in_mxn": net_return_in_mxn
    }

In [50]:

# Llamar a la función y asignar los valores retornados a variables
num_contracts = 1  # Ejemplo: se pueden ajustar los contratos según sea necesario
es_buyhold = simulate_buy_and_hold(es_history, symbol, fx_history, num_contracts)

# Asignar los valores retornados a variables específicas
es_buyhold_points = es_buyhold["return_in_points"]
es_buyhold_usd = es_buyhold["return_in_usd"]
es_buyhold_mxn = es_buyhold["return_in_mxn"]

# Imprimir fechas
print(f"Start Date: {es_history.index[0]}")
print(f"End Date: {es_history.index[-1]}")
# Imprimir los resultados
print(f"Return in Points: {es_buyhold_points}")
print(f"Return in USD: {es_buyhold_usd}")
print(f"Return in MXN: {es_buyhold_mxn}")


# Strategy 2: Risk measurement, risk target and capital


## Contract risk in base currency

Update summary dictionary to include the daily and annual contract risk 

risk in currency terms = notional exposure * std dev

notional exposure = last closing price * multiplier

In [49]:
# Added to helper.py, delete next time
def calculate_contract_risk(summary, instruments, timeseries):
    """
    Calculate daily and annualized contract risk for each symbol and update the summary dictionary.
    
    Inputs:
    summary: dict - the dictionary of summary statistics
    instruments: dict - the dictionary of instruments with metadata
    timeseries: dict - the dictionary of time series data
    
    Returns:
    summary: dict - the updated summary dictionary with contract risks
    """
    for symbol in summary.keys():
        multiplier = instruments.get(symbol, {}).get('multiplier', 1)
        last_closing_price = timeseries[symbol]['close'].iloc[-1]
        std_dev = summary[symbol]['std_dev'] / 100  # Convert percentage to decimal
        annualized_std_dev = summary[symbol]['annualized_std_dev'] / 100  # Convert percentage to decimal

        daily_contract_risk = multiplier * last_closing_price * std_dev
        annualized_contract_risk = multiplier * last_closing_price * annualized_std_dev

        summary[symbol]['daily_contract_risk'] = daily_contract_risk
        summary[symbol]['annualized_contract_risk'] = annualized_contract_risk

    return summary

In [50]:
# Update summary with contract risks

summary = calculate_contract_risk(summary, instruments, timeseries)
print("Summary Statistics for Each Symbol with Contract Risks:")
for symbol, stats in summary.items():
    print(f"{symbol}:")
    print(summary[symbol])

Summary Statistics for Each Symbol with Contract Risks:
ES:
{'mean_return': 0.038056750980877456, 'std_dev': 1.2504315625779854, 'annualized_mean': 9.742528251104629, 'annualized_std_dev': 20.006905001247766, 'daily_sharpe_ratio': 0.030434893135947997, 'annualized_sharpe_ratio': 0.48695829017516795, 'monthly_skew': -14.351501476816775, 'lower_fat_tail_ratio': 2.5680685342096266, 'higher_fat_tail_ratio': 1.7483440973176516, 'daily_contract_risk': 3333.02533005162, 'annualized_contract_risk': 53328.40528082592}
MES:
{'mean_return': 0.051487616191232134, 'std_dev': 1.3017453243703707, 'annualized_mean': 13.180829744955426, 'annualized_std_dev': 20.82792518992593, 'daily_sharpe_ratio': 0.039552756769943236, 'annualized_sharpe_ratio': 0.6328441083190918, 'monthly_skew': -7.616867247540757, 'lower_fat_tail_ratio': 2.125288636742835, 'higher_fat_tail_ratio': 1.4225590023635961, 'daily_contract_risk': 347.01275984403156, 'annualized_contract_risk': 5552.204157504505}
USDMXN:
{'mean_return': 0.

In [51]:
target_risk = .20

# Compute target risk in currency
target_risk_in_usd = capital * target_risk


In [52]:
# Compute position size
def calculate_position_size(target_risk_in_usd, contract_risk, capital):
    return target_risk_in_usd / contract_risk

In [53]:
# Calculate position size for each symbol
position_sizes = {}
for symbol, stats in summary.items():
    position_size = calculate_position_size(target_risk_in_usd, stats['annualized_contract_risk'], capital)
    position_sizes[symbol] = position_size
# Print position sizes
print ("Position sizes to achieve target risk per instrument:")


Position sizes to achieve target risk per instrument:
